In [1]:
import syft
from syft.core import torch
from syft.core.worker import LocalWorker, BaseWorker

hook = syft.core.torch_.hook.TorchHook()

local = hook.worker
remote = LocalWorker(id=1, hook=hook)

Hooking into Torch...
Overloading complete.


In [2]:
x = torch.zeros(2,2)

In [3]:
y = x.send_(remote)

In [4]:
z = y + y

Message:{'has_self': True, 'self': '_fl.5820520467', 'command': '__add__', 'args': ('_fl.5820520467',), 'kwargs': {}, 'arg_types': ['FloatTensor'], 'kwarg_types': []}
Compiled Result:{"registration": {"id": 9414612264, "owners": [1], "is_pointer": true}, "torch_type": "torch.FloatTensor", "var_data": null, "var_grad": null}


TypeError: register_object_() missing 1 required positional argument: 'obj'

In [5]:
ry = remote.objects[list(remote.objects.keys())[0]]
ry

[torch.FloatTensor - Locations:[1]]

In [7]:
z = y.get_()
z


 0  0
 0  0
[torch.FloatTensor of size 2x2]